# 05. 데이터 전처리 (Data Preprocessing)

## 학습 목표
- 결측치 처리 전략 이해
- 특성 스케일링 방법 비교
- 범주형 변수 인코딩
- 불균형 데이터 처리

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.datasets import load_iris, load_wine

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 1. 결측치 처리 (Handling Missing Values)

In [ ]:
# 결측치가 있는 샘플 데이터 생성
np.random.seed(42)
data = {
    'age': [25, 30, np.nan, 40, 35, np.nan, 50, 28],
    'income': [50000, np.nan, 60000, 80000, np.nan, 70000, 90000, 55000],
    'score': [85, 90, 75, np.nan, 88, 92, np.nan, 78]
}
df = pd.DataFrame(data)

print("원본 데이터:")
print(df)
print(f"\n결측치 개수:\n{df.isnull().sum()}")
print(f"\n결측치 비율:\n{df.isnull().mean() * 100:.2f}%")

### 1.1 SimpleImputer - 기본 대체 전략

In [ ]:
# 평균값으로 대체
imputer_mean = SimpleImputer(strategy='mean')
df_mean = pd.DataFrame(
    imputer_mean.fit_transform(df),
    columns=df.columns
)

# 중앙값으로 대체
imputer_median = SimpleImputer(strategy='median')
df_median = pd.DataFrame(
    imputer_median.fit_transform(df),
    columns=df.columns
)

# 최빈값으로 대체
imputer_frequent = SimpleImputer(strategy='most_frequent')
df_frequent = pd.DataFrame(
    imputer_frequent.fit_transform(df),
    columns=df.columns
)

# 상수값으로 대체
imputer_constant = SimpleImputer(strategy='constant', fill_value=0)
df_constant = pd.DataFrame(
    imputer_constant.fit_transform(df),
    columns=df.columns
)

print("평균값 대체:")
print(df_mean)
print(f"\n중앙값 대체 (age 컬럼): {df_median['age'].values}")
print(f"최빈값 대체 (age 컬럼): {df_frequent['age'].values}")

### 1.2 KNNImputer - K-최근접 이웃 대체

In [ ]:
# KNN 기반 결측치 대체
imputer_knn = KNNImputer(n_neighbors=3)
df_knn = pd.DataFrame(
    imputer_knn.fit_transform(df),
    columns=df.columns
)

print("KNN 대체:")
print(df_knn)

# 시각화 비교
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (method, df_filled) in zip(axes, [
    ('Mean', df_mean), 
    ('Median', df_median), 
    ('KNN', df_knn)
]):
    ax.scatter(df_filled['age'], df_filled['income'], alpha=0.7, s=100)
    ax.set_xlabel('Age')
    ax.set_ylabel('Income')
    ax.set_title(f'{method} Imputation')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. 특성 스케일링 (Feature Scaling)

In [ ]:
# 스케일이 다른 데이터 생성
np.random.seed(42)
data_scale = {
    'age': np.random.randint(20, 60, 100),
    'income': np.random.randint(30000, 150000, 100),
    'score': np.random.uniform(0, 100, 100)
}
df_scale = pd.DataFrame(data_scale)

print("원본 데이터 통계:")
print(df_scale.describe())

### 2.1 StandardScaler (표준화)

In [ ]:
# StandardScaler: (x - mean) / std
scaler_standard = StandardScaler()
df_standard = pd.DataFrame(
    scaler_standard.fit_transform(df_scale),
    columns=df_scale.columns
)

print("StandardScaler 결과:")
print(df_standard.describe())
print(f"\n평균: {df_standard.mean().values}")
print(f"표준편차: {df_standard.std().values}")

### 2.2 MinMaxScaler (정규화)

In [ ]:
# MinMaxScaler: (x - min) / (max - min)
scaler_minmax = MinMaxScaler(feature_range=(0, 1))
df_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(df_scale),
    columns=df_scale.columns
)

print("MinMaxScaler 결과:")
print(df_minmax.describe())
print(f"\n최솟값: {df_minmax.min().values}")
print(f"최댓값: {df_minmax.max().values}")

### 2.3 RobustScaler (이상치에 강건)

In [ ]:
# RobustScaler: (x - median) / IQR
scaler_robust = RobustScaler()
df_robust = pd.DataFrame(
    scaler_robust.fit_transform(df_scale),
    columns=df_scale.columns
)

print("RobustScaler 결과:")
print(df_robust.describe())

### 2.4 스케일러 비교 시각화

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

# 이상치 추가
df_outlier = df_scale.copy()
df_outlier.loc[0, 'income'] = 500000  # 이상치 추가

scalers = [
    ('Original', df_outlier),
    ('StandardScaler', pd.DataFrame(StandardScaler().fit_transform(df_outlier), columns=df_outlier.columns)),
    ('MinMaxScaler', pd.DataFrame(MinMaxScaler().fit_transform(df_outlier), columns=df_outlier.columns)),
    ('RobustScaler', pd.DataFrame(RobustScaler().fit_transform(df_outlier), columns=df_outlier.columns))
]

for ax, (name, data) in zip(axes, scalers):
    ax.boxplot([data['age'], data['income'], data['score']], labels=['age', 'income', 'score'])
    ax.set_title(name)
    ax.set_ylabel('Value')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 범주형 변수 인코딩 (Categorical Encoding)

In [ ]:
# 범주형 데이터 샘플
data_cat = {
    'color': ['red', 'blue', 'green', 'red', 'blue', 'green', 'red'],
    'size': ['S', 'M', 'L', 'M', 'S', 'L', 'M'],
    'quality': ['good', 'excellent', 'poor', 'good', 'excellent', 'poor', 'good']
}
df_cat = pd.DataFrame(data_cat)

print("범주형 데이터:")
print(df_cat)

### 3.1 LabelEncoder (레이블 인코딩)

In [ ]:
# LabelEncoder: 범주를 정수로 변환
le_color = LabelEncoder()
df_cat['color_encoded'] = le_color.fit_transform(df_cat['color'])

print("LabelEncoder 결과:")
print(df_cat[['color', 'color_encoded']])
print(f"\n클래스: {le_color.classes_}")
print(f"변환: {dict(zip(le_color.classes_, le_color.transform(le_color.classes_)))}")

### 3.2 OneHotEncoder (원-핫 인코딩)

In [ ]:
# OneHotEncoder: 범주를 이진 벡터로 변환
ohe = OneHotEncoder(sparse_output=False)
color_onehot = ohe.fit_transform(df_cat[['color']])

# DataFrame으로 변환
df_onehot = pd.DataFrame(
    color_onehot,
    columns=ohe.get_feature_names_out(['color'])
)

print("OneHotEncoder 결과:")
print(pd.concat([df_cat['color'], df_onehot], axis=1))

### 3.3 OrdinalEncoder (순서형 인코딩)

In [ ]:
# OrdinalEncoder: 순서가 있는 범주형 변수
oe = OrdinalEncoder(categories=[['poor', 'good', 'excellent']])
df_cat['quality_encoded'] = oe.fit_transform(df_cat[['quality']])

print("OrdinalEncoder 결과:")
print(df_cat[['quality', 'quality_encoded']])
print(f"\n순서: poor(0) < good(1) < excellent(2)")

### 3.4 Pandas get_dummies

In [ ]:
# pandas의 get_dummies (간편한 원-핫 인코딩)
df_dummies = pd.get_dummies(df_cat[['color', 'size']], prefix=['color', 'size'])

print("pd.get_dummies 결과:")
print(df_dummies.head())

# drop_first=True로 다중공선성 방지
df_dummies_drop = pd.get_dummies(df_cat[['color', 'size']], prefix=['color', 'size'], drop_first=True)
print(f"\ndrop_first=True (shape: {df_dummies_drop.shape}):")
print(df_dummies_drop.head())

## 4. 특성 선택 (Feature Selection)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Iris 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

print(f"원본 데이터: {X.shape}")
print(f"특성 이름: {iris.feature_names}")

### 4.1 SelectKBest (통계적 선택)

In [ ]:
# F-통계량 기반 선택
selector_f = SelectKBest(score_func=f_classif, k=2)
X_kbest_f = selector_f.fit_transform(X, y)

# 상호정보량 기반 선택
selector_mi = SelectKBest(score_func=mutual_info_classif, k=2)
X_kbest_mi = selector_mi.fit_transform(X, y)

print("SelectKBest (F-statistic):")
scores_f = pd.DataFrame({
    'Feature': iris.feature_names,
    'Score': selector_f.scores_
}).sort_values('Score', ascending=False)
print(scores_f)

print("\nSelectKBest (Mutual Information):")
scores_mi = pd.DataFrame({
    'Feature': iris.feature_names,
    'Score': selector_mi.scores_
}).sort_values('Score', ascending=False)
print(scores_mi)

### 4.2 RFE (재귀적 특성 제거)

In [ ]:
# RFE with Random Forest
estimator = RandomForestClassifier(n_estimators=50, random_state=42)
selector_rfe = RFE(estimator, n_features_to_select=2, step=1)
X_rfe = selector_rfe.fit_transform(X, y)

print("RFE 결과:")
rfe_result = pd.DataFrame({
    'Feature': iris.feature_names,
    'Selected': selector_rfe.support_,
    'Ranking': selector_rfe.ranking_
}).sort_values('Ranking')
print(rfe_result)

### 4.3 특성 중요도 (랜덤 포레스트)

In [ ]:
# 랜덤 포레스트 특성 중요도
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

importance = pd.DataFrame({
    'Feature': iris.feature_names,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(importance['Feature'], importance['Importance'])
plt.xlabel('Importance')
plt.title('Random Forest Feature Importance - Iris Dataset')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. 불균형 데이터 처리 (Imbalanced Data)

In [ ]:
from sklearn.datasets import make_classification

# 불균형 데이터 생성 (10:1 비율)
X_imb, y_imb = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    weights=[0.9, 0.1],  # 90% vs 10%
    random_state=42
)

# 클래스 분포 확인
unique, counts = np.unique(y_imb, return_counts=True)
print("클래스 분포:")
for cls, cnt in zip(unique, counts):
    print(f"  Class {cls}: {cnt} ({cnt/len(y_imb)*100:.1f}%)")

# 시각화
plt.figure(figsize=(8, 5))
plt.bar(['Class 0', 'Class 1'], counts, color=['skyblue', 'salmon'])
plt.ylabel('Count')
plt.title('Imbalanced Dataset Distribution')
plt.grid(True, alpha=0.3)
plt.show()

### 5.1 SMOTE 개념 (이론)

In [ ]:
# SMOTE (Synthetic Minority Over-sampling Technique) 개념 설명
print("""
SMOTE 작동 원리:

1. 소수 클래스의 각 샘플에 대해:
   - K개의 최근접 이웃을 찾음 (보통 k=5)
   
2. 랜덤하게 선택된 이웃과의 선형 보간:
   - new_sample = sample + λ × (neighbor - sample)
   - λ는 0과 1 사이의 랜덤값
   
3. 합성 샘플을 생성하여 소수 클래스 증강

장점:
- 과적합 위험이 낮음 (단순 복제가 아님)
- 결정 경계가 더 일반화됨

단점:
- 노이즈에 민감할 수 있음
- 고차원 데이터에서는 효과가 제한적

사용 방법:
- pip install imbalanced-learn
- from imblearn.over_sampling import SMOTE
- smote = SMOTE(random_state=42)
- X_resampled, y_resampled = smote.fit_resample(X, y)
""")

### 5.2 클래스 가중치 조정

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(
    X_imb, y_imb, test_size=0.3, random_state=42
)

# 가중치 없음
clf_no_weight = LogisticRegression(random_state=42, max_iter=1000)
clf_no_weight.fit(X_train_imb, y_train_imb)
y_pred_no_weight = clf_no_weight.predict(X_test_imb)

# 가중치 조정 (balanced)
clf_balanced = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
clf_balanced.fit(X_train_imb, y_train_imb)
y_pred_balanced = clf_balanced.predict(X_test_imb)

print("=== 가중치 없음 ===")
print(classification_report(y_test_imb, y_pred_no_weight))

print("\n=== 가중치 조정 (balanced) ===")
print(classification_report(y_test_imb, y_pred_balanced))

## 6. 실전 전처리 파이프라인

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# 혼합 데이터 생성
data_mixed = {
    'age': [25, np.nan, 35, 40, 30, 45, np.nan, 28],
    'income': [50000, 60000, np.nan, 80000, 70000, 90000, 55000, np.nan],
    'city': ['Seoul', 'Busan', 'Seoul', 'Daegu', 'Busan', 'Seoul', 'Daegu', 'Busan'],
    'education': ['Bachelor', 'Master', 'PhD', 'Bachelor', 'Master', 'PhD', 'Bachelor', 'Master'],
    'purchased': [0, 1, 1, 0, 1, 1, 0, 1]
}
df_mixed = pd.DataFrame(data_mixed)

X_mixed = df_mixed.drop('purchased', axis=1)
y_mixed = df_mixed['purchased']

print("혼합 데이터:")
print(df_mixed)

In [ ]:
# 수치형/범주형 특성 분리
numeric_features = ['age', 'income']
categorical_features = ['city', 'education']

# 수치형 전처리 파이프라인
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 전처리 파이프라인
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# ColumnTransformer로 결합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 전체 파이프라인
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42))
])

# 학습 (작은 데이터이므로 전체 사용)
pipeline.fit(X_mixed, y_mixed)

# 새로운 데이터 예측
new_data = pd.DataFrame({
    'age': [30],
    'income': [70000],
    'city': ['Seoul'],
    'education': ['Master']
})

prediction = pipeline.predict(new_data)
probability = pipeline.predict_proba(new_data)

print(f"\n예측 결과: {prediction[0]}")
print(f"확률: {probability[0]}")

## 정리

### 핵심 개념

**결측치 처리:**
- **SimpleImputer**: 평균, 중앙값, 최빈값, 상수로 대체
- **KNNImputer**: K-최근접 이웃 기반 대체

**특성 스케일링:**
- **StandardScaler**: 평균 0, 표준편차 1 (정규분포 가정)
- **MinMaxScaler**: 0-1 범위로 정규화
- **RobustScaler**: 중앙값과 IQR 사용 (이상치에 강건)

**범주형 인코딩:**
- **LabelEncoder**: 순서 없는 분류 (타겟 변수용)
- **OneHotEncoder**: 이진 벡터로 변환 (다중공선성 주의)
- **OrdinalEncoder**: 순서가 있는 범주형

**불균형 데이터:**
- **SMOTE**: 합성 샘플 생성 (요구사항: imbalanced-learn)
- **class_weight**: 모델 가중치 조정

### 다음 단계
- Pipeline과 ColumnTransformer 활용
- 교차 검증과 전처리 통합
- 실전 프로젝트 적용